In [1]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders

import pandas as pd
import numpy as np
from clickhouse_driver import Client
import clickhouse_connect as cc
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import tqdm
from datetime import datetime, timedelta
import time

In [2]:
ds = datetime.now() + timedelta(hours=5)

In [3]:
import yaml
with open('C:\\Users\\ngro\\Jupyter Files\\pass.yaml', 'r') as stream:
    try:
        parsed_yaml=yaml.safe_load_all(stream)
        for key in parsed_yaml:
            pass_dict = key
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
# Connection to DB
client = cc.get_client(host=pass_dict['clickhouse_host'], port=pass_dict['clickhouse_port'], username='nikolai_gromov', password=pass_dict['personal_clickhouse'])

In [6]:
#Getting data
query = f'''
    SELECT 1 as example
        '''
df_res = client.query_df(query)
df_res

,example
0,1


In [7]:
# Email header
emailfrom = pass_dict['email']
password = pass_dict['email_password']
msg = MIMEMultipart()
msg["From"] = emailfrom
msg["To"] = 'example@receiver.kz'
msg['Subject'] = f'Клиенты в просрочке 1-30 по {ds}'
msg.attach(MIMEText('Текст письма'))

# Attaching excel-file
df_res.to_excel('somedata.xlsx')
part = MIMEBase('application', "octet-stream")
part.set_payload(open("somedata.xlsx", "rb").read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', 'attachment; filename="somedata.xlsx"')
msg.attach(part)

#context = ssl.SSLContext(ssl.PROTOCOL_SSLv3)
#SSL connection only working on Python 3+
smtp = smtplib.SMTP("smtp.gmail.com:587")
smtp.starttls()
smtp.login(emailfrom,password)
smtp.sendmail(emailfrom, ['example@receiver.kz'], msg.as_string())
smtp.quit()
print('email success')

email success
